In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/fenago/datasets/main/Bias_correction_ucl.csv")

In [6]:
df.sample(2)

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
590,16.0,26.4,24.3,67.874313,96.791023,27.404923,22.926537,12.462894,91.130902,0.620191,...,1.235356,0.0,0.0,37.4697,126.995,82.2912,2.2579,5575.939453,28.2,22.0
1277,3.0,32.0,23.2,39.381092,75.439720,34.711694,24.901418,6.532692,12.430460,0.152634,...,0.000000,0.0,0.0,37.5776,127.058,33.3068,0.2661,4833.543945,32.8,24.3


In [4]:
df.drop('Date', inplace=True, axis=1)
df.dropna(inplace=True)

In [5]:
df.dtypes

station             float64
Present_Tmax        float64
Present_Tmin        float64
LDAPS_RHmin         float64
LDAPS_RHmax         float64
LDAPS_Tmax_lapse    float64
LDAPS_Tmin_lapse    float64
LDAPS_WS            float64
LDAPS_LH            float64
LDAPS_CC1           float64
LDAPS_CC2           float64
LDAPS_CC3           float64
LDAPS_CC4           float64
LDAPS_PPT1          float64
LDAPS_PPT2          float64
LDAPS_PPT3          float64
LDAPS_PPT4          float64
lat                 float64
lon                 float64
DEM                 float64
Slope               float64
Solar radiation     float64
Next_Tmax           float64
Next_Tmin           float64
dtype: object

In [7]:
target = df[['Next_Tmax', 'Next_Tmin']]
features = df.drop(['Next_Tmax', 'Next_Tmin'], axis=1)

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
feature_array = scaler.fit_transform(features)
features = pd.DataFrame(feature_array, columns=features.columns)

In [9]:
features.sample(4)

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation
283,0.416667,0.335227,0.709677,0.817844,0.868221,0.376193,0.581919,0.268928,0.249612,0.868286,...,0.922873,0.001680,0.110920,0.265996,0.052229,0.429025,0.838188,0.081664,0.103313,0.868146
2768,0.416667,0.215909,0.483871,0.401006,0.824690,0.549085,0.499525,0.057489,0.236813,0.385140,...,0.608175,0.000133,0.000000,0.000000,0.000000,0.429025,0.838188,0.081664,0.103313,0.327631
4890,0.041667,0.590909,0.612903,0.440379,0.846889,0.559389,0.571153,0.134253,0.312821,0.513373,...,0.409941,0.000370,0.000000,0.000000,0.000000,0.786017,0.666667,0.161990,0.081816,0.856329
4741,0.083333,0.403409,0.467742,0.452254,0.680443,0.481718,0.515963,0.079469,0.099890,0.622002,...,0.364830,0.000000,0.000018,0.000000,0.000000,0.643008,0.750809,0.104702,0.032994,0.892347


In [10]:
model = tf.keras.Sequential()

In [11]:
model.add(tf.keras.layers.InputLayer(input_shape=(features.shape[1],),name='Input_layer'))

In [12]:
model.add(tf.keras.layers.Dense(2, name='Output_layer'))

In [13]:
model.compile(tf.optimizers.RMSprop(0.001), loss='mse')

In [14]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [15]:
model.fit(x=features.to_numpy(), y=target.to_numpy(),epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50
  1/238 [..............................] - ETA: 0s - loss: 705.6977WARNING:tensorflow:From /opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
238/238 [==============================] - 0s 2ms/step - loss: 669.8055
Epoch 2/50
238/238 [==============================] - 0s 1ms/step - loss: 565.1844
Epoch 3/50
238/238 [==============================] - 0s 2ms/step - loss: 470.3730
Epoch 4/50
238/238 [==============================] - 0s 2ms/step - loss: 385.0522
Epoch 5/50
238/238 [==============================] - 0s 1ms/step - loss: 309.3490
Epoch 6/50
238/238 [==============================] - 0s 1ms/step - loss: 243.2048
Epoch 7/50
238/238 [==============================] - 0s 2ms/step - loss: 186.6502
Epoch 8/50
238/238 [==============================] - 0s 1ms/step - loss: 

In [16]:
loss = model.evaluate(features.to_numpy(), target.to_numpy())
print('loss:', loss)

238/238 [==============================] - 0s 997us/step - loss: 3.5634
loss: 3.563430070877075


In [17]:
tensorboard –-logdir=logs/

SyntaxError: invalid character in identifier (<ipython-input-17-bca727cf9d7e>, line 1)